# Retrieving Data from OpenStreetMap

## Introduction to OpenStreetMap

[OpenStreetMap (OSM)](https://openstreetmap.org) represents a comprehensive, collaborative project aimed at creating a freely editable map of the world. This initiative is a testament to a global effort in gathering and sharing geospatial data on a scale that covers our environment's various aspects, such as streets, buildings, services, and land use, to name a few. At its core, OSM is supported by a community that exceeds 8 million registered users, contributing approximately 4 million updates every day. This data, which consists of over 7 billion nodes, facilitates a range of applications beyond mapping, including routing, geocoding, education, and research, significantly impacting humanitarian efforts and economic development.

- **Explore More:** [OSM Wiki](https://wiki.openstreetmap.org/wiki/Main_Page)

### Contribute to OpenStreetMap

Anyone can contribute to this rich database by adding new data, correcting, or improving the existing information.

- **Get Involved:** [How to contribute](https://wiki.openstreetmap.org/wiki/Main_Page)

### Applications of OSM Data

OSM's utility extends beyond its primary function as a mapping tool. It is instrumental in humanitarian projects, particularly in crisis response and promoting economic development in underserved regions.

- **Discover:** [Humanitarian OpenStreetMap Team (HOTOSM)](https://www.hotosm.org)

## Essential Tools for This Lesson

### OSMnx

This section introduces [OSMnx](https://github.com/gboeing/osmnx), a Python package designed for retrieving, constructing, analyzing, and visualizing street networks from OpenStreetMap data. OSMnx simplifies the process of downloading street networks and points of interest, supporting various applications like route finding across different modes of transportation.

- **Read:** Boeing, G. 2017. ["OSMnx: New Methods for Acquiring, Constructing, Analyzing, and Visualizing Complex Street Networks."](https://www.researchgate.net/publication/309738462_OSMnx_New_Methods_for_Acquiring_Constructing_Analyzing_and_Visualizing_Complex_Street_Networks) Computers, Environment and Urban Systems 65, 126-139. doi:10.1016/j.compenvurbsys.2017.05.004
- **Tutorial:** [OSMnx Overview](https://github.com/gboeing/osmnx-examples/blob/master/notebooks/01-overview-osmnx.ipynb)

### NetworkX

We will also leverage [NetworkX](https://networkx.github.io/documentation//), a powerful Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks, particularly those derived from OSM data.

## Downloading and Visualizing OpenStreetMap Data with OSMnx

OSMnx offers an intuitive interface for accessing OpenStreetMap data through the OverPass API, enabling users to download and visualize street networks and additional geospatial information for specified areas of interest.

### Street Network

Utilizing the [`osmnx.graph` module](https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.graph), users can construct routable road network graphs based on various criteria such as place names, bounding boxes, or polygons. For example, we will explore how to fetch data for the around Stora Torget in Karlstad, Sweden using a place name query, which relies on the Nominatim Geocoding API for location lookups.

- **Function Highlight:** [graph_from_place()](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.graph.graph_from_place)


In [ ]:
import osmnx 

# Geocode the location to get its latitude and longitude
location_point = osmnx.geocoder.geocode("Stora Torget, Karlstad, Sweden")

# Define the distance by which to extend the bounding box (in meters)
distance = 1000  # Extend by 500 meters

# Calculate the bounding box around the location, extended by the specified distance
north, south, east, west = osmnx.utils_geo.bbox_from_point(
    point=location_point, dist=distance)

# You can now use this bounding box to get the graph
graph = osmnx.graph_from_bbox(north, south, east, west, network_type='all')



print(f"Bounding Box: North: {north}, South: {south}, East: {east}, West: {west}")


In [ ]:
type(graph)

The type is a networkx.MultiDiGraph object.

OSMnx’s graphs do not have a built-in method to plot them, but the package comes with a function to do so:

In [ ]:
fig, ax = osmnx.plot_graph(osmnx.project_graph(graph))

 Osmnx.plot_graph() uses matplotlib. The function returns a (figure, axes) tuple, that can be used to modify the figure using all matplotlib functions we already got to know.

We can see that our graph contains nodes (the points) and edges (the lines) that connects those nodes.

### Convert a graph to `GeoDataFrame`s

The street network we just downloaded is a *graph*, more specifically a
`networkx.MultiDiGraph`. Its main purpose is to represent the topological
relationships between nodes and the links (edges) between them. Sometimes, it
is more convenient to have the underlying geodata in `geopandas.GeoDataFrame`s.
OSMnx comes with a convenient function that converts a graph into two geo-data
frames, one for nodes, and one for edges:
[`osmnx.graph_to_gdfs()`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.utils_graph.graph_to_gdfs).

In [ ]:
nodes, edges = osmnx.graph_to_gdfs(graph)

In [ ]:
nodes.head()

In [ ]:
edges.head()


### Place polygon

Let’s also plot the polygon that represents our area of interest (Stora Torget, Karlstad). We can retrieve the polygon geometry using the
using the bounding box as input to geopandas.

In [ ]:
import geopandas
from shapely.geometry import Polygon

# Assuming north, south, east, west variables are already defined

# Create a polygon from the bounding box
polygon = Polygon([(west, north), (east, north), (east, south), (west, south)])

# Create a GeoDataFrame from the polygon
area = geopandas.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[polygon])

# Now `area_gdf` represents the area of the bounding box as a GeoDataFrame


In [ ]:
# Check the data type
type(area)

In [ ]:
# Check data values
area

In [ ]:
# Plot the area:
area.plot()

### Building footprints

Besides network data, OSMnx can also download any other data contained in the OpenStreetMap database. This includes, for instance, building footprints, and different points-of-interests (POIs). To download arbitrary geometries, filtered by [OSM tags](https://wiki.openstreetmap.org/wiki/Map_features) and a place name, use [`osmnx.geometries_from_place()`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.geometries.geometries_from_place). The tag to retrieve all [buildings](https://wiki.openstreetmap.org/wiki/Buildings) is `building = yes`.

In [ ]:
# Download building geometries within the specified bounding box
buildings = osmnx.geometries.geometries_from_bbox(north, south, east, west, tags={"building": True})

In [ ]:

len(buildings) 

In [ ]:
buildings.head() 

As you can see, there are several columns in `buildings`. Each column contains
information about a specific tag that OpenStreetMap contributors have added.
Each tag consists of a key (the column name), and a values (for example
`building=yes` or `building=school`). Read more about tags and tagging
practices in the [OpenStreetMap
wiki](https://wiki.openstreetmap.org/wiki/Tags).

In [ ]:
buildings.columns 

In [ ]:
buildings.plot()

### Points-of-interest

Point-of-interest (POI) is a generic concept that describes point locations
that represent places of interest. As `osmnx.geometries_from_place()` can download any geometry data contained in the OpenStreetMap database, it can also be used to download any kind of POI data. 


In OpenStreetMap, many POIs are described using the [`amenity`
tag](https://wiki.openstreetmap.org/wiki/Key:amenity).  We can, for example,
retrieve all restaurant locations by querying `amenity=restaurant`.

In [ ]:
# Download restaurant geometries within the specified bounding box
restaurants = osmnx.geometries_from_bbox(north, south, east, west, tags={"amenity": "restaurant"})

# Print the number of restaurants found
print(len(restaurants))


In [ ]:
# Available columns
restaurants.columns.values 

In [ ]:
# Select some useful cols and print
interesting_columns = [
    "name",
    "opening_hours",
    "addr:city",
    "addr:country",
    "addr:housenumber",
    "addr:postcode",
    "addr:street"
]

# Print only selected cols
restaurants[interesting_columns].head(10) 


### Parks and green areas

Let’s try to fetch all public parks in the Kamppi area. In OpenStreetMap,
[parks hould be tagged](https://wiki.openstreetmap.org/wiki/Map_features) as
`leisure = park`.  Smaller green areas (*puistikot*) are sometimes also tagged
`landuse = grass`. We can combine multiple tags in one data query.

In [ ]:

# Assuming north, south, east, west are already defined

# Download geometries tagged as parks within the specified bounding box
parks =  osmnx.geometries_from_bbox(north, south, east, west, tags={"leisure": "park"})

print(len(parks))


In [ ]:
parks.head()

In [ ]:
parks.plot(color="green") 

### Plotting the data

Let’s create a map out of the streets, buildings, restaurants, and the area polygon.

In [ ]:
import matplotlib
figure, ax = matplotlib.pyplot.subplots(figsize=(12,8))

# Plot the footprint
area.plot(ax=ax, facecolor="black")

# Plot parks
parks.plot(ax=ax, facecolor="green")

# Plot street ‘edges’
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

# Plot buildings
buildings.plot(ax=ax, facecolor="silver", alpha=0.7)

# Plot restaurants
restaurants.plot(ax=ax, color="yellow", alpha=0.7, markersize=10)

## Sources

This lesson is inspired and has adapted or reused material from University of Helsinki Automating GIS processis course (https://autogis-site.readthedocs.io/en/latest/course-info/license.html) under a Creative Commons Attribution-ShareAlike 4.0 International licence (https://creativecommons.org/licenses/by-sa/4.0/deed.en).